<h1>Import</h1>

In [10]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

<h1>Load image dir</h1>

In [2]:
img_folder_dir = "./Images/Tin"
list_img_name = os.listdir(img_folder_dir)
list_img_dir = []
for img_name in list_img_name:
    list_img_dir.append(os.path.join(img_folder_dir, img_name))

<h1>Define support functions</h1>

In [3]:
def load_image(img_dir):
    return cv2.imread(img_dir)

def crop_edge(image, crop_x):
    # Get image dimensions
    if len(image.shape) == 3:
        height, width, _ = image.shape
    else:
        height, width = image.shape

    # Ensure X is not larger than half of width or height
    crop_x = min(crop_x, width // 2, height // 2)

    # Crop the image (remove X pixels from each edge)
    cropped_image = image[crop_x:height - crop_x, crop_x:width - crop_x]

    return cropped_image

def convert_gray(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

def convert_invert(image):
    return cv2.bitwise_not(image)        

def adjust_bright(image, increase=1):
    mean_intensity_cv2 = cv2.mean(image)[0]
    bright = cv2.add(image, mean_intensity_cv2 * increase)
    return bright

def show_hist(image, title):
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    plt.figure(figsize=(4, 3))
    plt.plot(hist, color='black')
    plt.title(title)
    plt.xlabel("Pixel Intensity")
    plt.ylabel("Frequency")
    plt.xlim([0, 256])
    plt.grid()
    plt.show()

<h1>Define filter based threahold</h1>
- <b><code>increase</code></b> parameter can be tuned

In [4]:
def filter_threadhold(image, increase=1):
    mean_intensity_cv2 = cv2.mean(image)[0]
    mean_intensity_cv2 = min(mean_intensity_cv2 * increase, 255) #+10%
    _, thresh = cv2.threshold(image, mean_intensity_cv2, 255, cv2.THRESH_BINARY)
    filtered_image = np.where(image >= mean_intensity_cv2, image, 0).astype(np.uint8)
    return filtered_image

<h1>Define filter sobel to detect edge</h1>

In [5]:
def filter_sobel(image, kernel):
       
    # Apply Sobel filter in X and Y direction
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=kernel)  # X-direction
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=kernel)  # Y-direction

    # Compute the gradient magnitude
    sobel_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    sobel_magnitude = np.uint8(255 * sobel_magnitude / np.max(sobel_magnitude))  # Normalize

    return sobel_magnitude

<h1>Define function to find the largest area after apply Sobel filter</h1>

In [6]:
def find_largest_area(image, image_threadhold):
    # Convert to binary image (thresholding)
    _, binary = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Select the largest contour
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Create a mask for the largest contour
    mask = np.zeros_like(image)
    cv2.drawContours(mask, [largest_contour], -1, 255, thickness=cv2.FILLED)
    
    # # Apply mask on the original image
    # result = cv2.bitwise_and(image_threadhold, image_threadhold, mask=mask)
    
    return mask

<h1>Define function to apply the mask on image</h1>

In [293]:
def mask_largest_area(image, largest_area, top_cut=False, color=(0,255,0)):
    # Set mask color (Red in BGR format)
    mask_colored = cv2.cvtColor(largest_area, cv2.COLOR_GRAY2BGR)
    mask_colored[np.where(largest_area == 255)] = color  # Red overlay

    mask_h = mask_colored.shape[0]
    aligned_mask = np.zeros_like(image, dtype=np.uint8)

    # Place the mask at the top (y=0) and keep the width the same
    if top_cut == None:
        aligned_mask[:, :, :] = mask_colored  # Copy the mask into the top part
    elif top_cut:
        aligned_mask[-mask_h:, :, :] = mask_colored
    else:
        aligned_mask[:mask_h, :, :] = mask_colored  # Copy the mask into the top part

    # Blend the mask and image (Alpha blending)
    alpha = 0.5  # Transparency factor
    overlay = cv2.addWeighted(image, 1, aligned_mask, alpha, 0)
    return overlay

<h1>Define function for bounding box (Find & Apply mask)</h1>

In [240]:
def find_bbox_largest_area(largest_area):
    # Find contours in the mask
    contours, _ = cv2.findContours(largest_area, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw bounding rectangles around each detected object
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)  # Get bounding box

    return [x, y, w, h]

def mask_bbox(masked_result, bbox_largest_area, cut_pixel=0, color=(0,255,0)):
    x, y, w, h = bbox_largest_area

    img_h, img_w, _ = masked_result.shape

    y = y + cut_pixel
    
    cv2.rectangle(masked_result, (x, y), (x + w, y + h), color, 1)  # Draw rectangle
    return masked_result

<h1>Define function to approximate caculate the length and area of Mask or Bounding box</h1>

In [16]:
def cal_length_area(largest_area, bbox_largest_area, ratio):
    area = np.count_nonzero(largest_area)
    x, y, w, h = bbox_largest_area
    length = h * ratio
    return {
        "area": area, 
        "length": length
    }

<h1>Main process</h1>

In [378]:
length_area_liver = pd.DataFrame(columns=['img_dir','length','area'])
length_area_intestine = pd.DataFrame(columns=['img_dir','length','area'])

In [383]:
for img_dir in tqdm(list_img_dir):
    # img_dir = "./Images/Tin/242.b0.1.20250108_frame0000198_obj_8.png"
    image = load_image(img_dir)
    image = crop_edge(image, 1)
    image_gray = convert_gray(image)
    image_invert = convert_invert(image_gray)
    
    image_threadhold = filter_threadhold(image_invert, increase=1.2)
    
    image_sobel = filter_sobel(image_threadhold, kernel=1)

    largest_area = find_largest_area(image_sobel, image_threadhold)

    bbox_largest_area = find_bbox_largest_area(largest_area)
    
    masked_result = mask_largest_area(image, largest_area, top_cut=None)
    masked_result = mask_bbox(masked_result, bbox_largest_area)

    length_pixel_ratio = 2 / max(image.shape) # 2cm / 100 pixel ~ 0.02cm per pixel 
    
    info_length_area = cal_length_area(largest_area, bbox_largest_area, ratio=length_pixel_ratio)

    length_area_liver = pd.concat([length_area_liver, pd.DataFrame([{
        "img_dir": img_dir,
        "length": info_length_area["length"],
        "area": info_length_area["area"]
    }])], ignore_index=True)


    # Get image height and width
    img_h, img_w = image_invert.shape  # (height, width, channels)
    top_cut = False
    cut_pixel = 0
    x, y, w, h = bbox_largest_area
    # Determine cropping based on bbox position
    if y + h / 2 < img_h / 2:
        # BBox is in the top half → Keep only the lower half (crop below bbox)
        image_cut = image_invert[y + h :, :]
        cut_pixel = y + h
        top_cut = True
    else:
        # BBox is in the lower half → Keep only the upper half (crop above bbox)
        image_cut = image_invert[:y, :]

    image_cut_threadhold = filter_threadhold(image_cut, increase=1.05)
    image_cut_sobel = filter_sobel(image_cut_threadhold, kernel=1)
    cut_largest_area = find_largest_area(image_cut_sobel, image_threadhold)
    cut_bbox_largest_area = find_bbox_largest_area(cut_largest_area)
    
    cut_masked_result = mask_largest_area(masked_result, cut_largest_area, top_cut=top_cut, color=(255,0,0))
    cut_masked_result = mask_bbox(cut_masked_result, cut_bbox_largest_area, cut_pixel=cut_pixel, color=(255,0,0))    

    cut_info_length_area = cal_length_area(cut_largest_area, cut_bbox_largest_area, ratio=length_pixel_ratio)

    length_area_intestine = pd.concat([length_area_intestine, pd.DataFrame([{
        "img_dir": img_dir,
        "length": cut_info_length_area["length"],
        "area": cut_info_length_area["area"]
    }])], ignore_index=True)

    cv2.imwrite(os.path.join("./Images/Results", img_dir.split("/")[-1]), cut_masked_result)

    length_area_liver.to_csv('./Images/Results/length_area_liver.csv', index=False)
    length_area_intestine.to_csv('./Images/Results/length_area_intestine.csv', index=False)
    # break

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 253.55it/s]
